# 1.2.0: Add `Event`s

In `AudibleLight`, `Event` objects represent sounds placed within a `Scene`. There are two types of `Event` objects:
- Static `Event`s, which occupy the same point in space;
- Moving `Event`s, which move through space according to a particular trajectory (e.g., linear, random...)

`AudibleLight` defines a comprehensive API for adding `Event` objects to a `Scene`, as well as controlling parameters relating to the underlying audio file (e.g., duration, offset, any augmentations), how it moves through space (e.g., its velocity and resolution), and how it relates to the whole `Scene` (e.g., how loud it is versus the noise floor).

Note that the `Augmentation` API for `AudibleLight` has its own tutorial.

## Adding static `Event` objects

In [1]:
from audiblelight.core import Scene
from audiblelight import utils

In [2]:
scene = Scene(
    duration=60,
    sample_rate=44100,
    backend="rlr",
    backend_kwargs=dict(
        mesh=utils.get_project_root() / "tests/test_resources/meshes/Oyens.glb"
    ),
    fg_path=utils.get_project_root() / "tests/test_resources/soundevents",
)
scene.add_microphone(microphone_type="ambeovr")

CreateContext: Context created
CreateContext: Context created


We are now ready to add an `Event` to our scene. By default, this will start from between 0 and 59 seconds (i.e., `scene.duration` - 1), with an audio file pulled from our `fg_path`.

In [3]:
added = scene.add_event(event_type="static", alias="my_first_event")

CreateContext: Context created


2025-10-07 16:33:48.707 | INFO     | audiblelight.core:add_event:835 - Event added successfully: Static 'Event' with alias 'my_first_event', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/musicInstrument/3471.wav' (unloaded, 0 augmentations), 1 emitter(s).


Note that the same functionality could be achieved by calling `Scene.add_event_static` with the same arguments (minus `event_type="static"`).

### Using overrides

To control more parameters of the `Event`, we can pass in arguments to `add_event`. These arguments will **override** any distributions passed to `Scene.__init__`, allowing for more finegrained placing of events.

In [4]:
added2 = scene.add_event(
    event_type="static",
    alias="my_second_event",
    filepath=utils.get_project_root() / "tests/test_resources/soundevents/music/000010.mp3",
    duration=10,
    scene_start=5.0,
    event_start=2.0,
    polar=True,
    position=[-45., 0., 0.5]
)

CreateContext: Context created


2025-10-07 16:33:48.990 | INFO     | audiblelight.core:add_event:835 - Event added successfully: Static 'Event' with alias 'my_second_event', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/music/000010.mp3' (unloaded, 0 augmentations), 1 emitter(s).


This example loads in a music file with an offset at 2 seconds and a duration of 10 seconds (i.e., seconds 2 -- 12 will be used). The audio file will start 5 seconds in to the start of the scene. It will be placed 45 degrees to the front-left of the mic, level with the mic, and 0.5 meters away.

Note that, when dealing with multiple microphones added to a single `Scene`, as well as passing `polar=True`, we also need to pass the alias of the microphone to `mic=...`, so the correct offset can be calculated.

In [5]:
scene.clear_microphones()
mic1 = scene.add_microphone(microphone_type="monocapsule", alias="mic_a")
mic2 = scene.add_microphone(microphone_type="monocapsule", alias="mic_b")

pol = scene.add_event(
    event_type="static",
    polar=True,
    mic="mic_a",
    alias="pol",
    position=[-45., 0., 0.5]
)

CreateContext: Context created


CreateContext: Context created
CreateContext: Context created


2025-10-07 16:33:50.119 | INFO     | audiblelight.core:add_event:835 - Event added successfully: Static 'Event' with alias 'pol', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/waterTap/95709.wav' (unloaded, 0 augmentations), 1 emitter(s).


CreateContext: Context created


Now, the `pol` `Event` is placed so that it is -45 degrees from the `mic_a` `MicArray`.

In [6]:
scene.clear_microphone("mic_b")

CreateContext: Context created


### Inspecting `Event` objects

For more information on the `Event`, we can use its alias to grab it from the `Scene`

In [7]:
event2 = scene.get_event("pol")
print(event2)

Static 'Event' with alias 'pol', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/waterTap/95709.wav' (unloaded, 0 augmentations), 1 emitter(s).


We can also load the audio file with `Event.load_audio`.

Note that this will happen automatically whenever `scene.generate` is called, so you don't need to worry about making this part of your data generation code.

In [8]:
audio = event2.load_audio(ignore_cache=True, normalize=True)
print(event2.is_audio_loaded)

True


### Adding moving events

Simple moving `Event`s can be added in the same way as simple static `Event`s.

In [9]:
scene.clear_events()
moving1 = scene.add_event(event_type="moving", alias="my_first_moving_event")

CreateContext: Context created


2025-10-07 16:33:55.648 | INFO     | audiblelight.core:add_event:835 - Event added successfully: Moving 'Event' with alias 'my_first_moving_event', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/laughter/16204.wav' (unloaded, 0 augmentations), 3 emitter(s).


CreateContext: Context created


Moving `Event`s expose the same parameters involved when creating a static `Event` (e.g., duration, offset), along with some new ones.

In particular, we can control:
- the spatial velocity: how fast the event moves, in metres-per-second
- the spatial resolution: how many IRs are created per second, in Hz
- the trajectory: either `linear`, `circular`, or `random`
- the starting position of the event.

In [10]:
added2 = scene.add_event(
    event_type="moving",
    alias="my_second_moving_event",
    spatial_velocity=1.,
    spatial_resolution=0.5,
    shape="linear",
)

2025-10-07 16:34:15.558 | INFO     | audiblelight.core:add_event:835 - Event added successfully: Moving 'Event' with alias 'my_second_moving_event', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/waterTap/95709.wav' (unloaded, 0 augmentations), 4 emitter(s).


CreateContext: Context created


### A note on class labels

By default, `Event` objects will try and define a `class_id` and `class_label` attribute using the 13 classes of the [DCASE challenge.](https://dcase.community/challenge2024/task-audio-and-audiovisual-sound-event-localization-and-detection-with-source-distance-estimation)

`AudibleLight` will attempt to extract these attributes from the filepath of the audio file if they are not passed when creating an `Event`. Alternatively, if only one parameter is passed (e.g., just `class_label`), the missing attribute will be inferred from this using the DCASE classes.

Of course, these IDs and labels can also be overridden:

In [11]:
added = scene.add_event(event_type="static", alias="my_custom_event", class_id=100, class_label="customClass")
print(added.class_label, added.class_id)

2025-10-07 16:34:16.016 | INFO     | audiblelight.core:add_event:835 - Event added successfully: Static 'Event' with alias 'my_custom_event', audio file '/home/huw-cheston/Documents/python_projects/AudibleLight/tests/test_resources/soundevents/telephone/30085.wav' (unloaded, 0 augmentations), 1 emitter(s).


CreateContext: Context created
customClass 100


and any metadata generated from the scene (e.g., `generate_dcase2024_metadata`) will adhere to the custom IDs.